Bibliography:

* [Stacked Convolutional Auto-Encoders for Hierarchical Feature Extraction](http://people.idsia.ch/~ciresan/data/icann2011.pdf)
    

In [1]:
import torch
import torchvision
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from torchvision import datasets
from torchvision.utils import save_image

import skimage 
import math
# import io
# import requests
# from PIL import Image

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import sys
import os


In [2]:
import cae

In [3]:
%load_ext autoreload
%autoreload

In [4]:
from helpers import *
from helper_modules import *
from multi_res_cae import *

In [5]:
# frcae = MultiFullCAE((640,480))

In [6]:
# mrcae = MultiResCAE([640,480])

In [7]:
# Hyper Parameters
# num_epochs = 5
# batch_size = 100
# learning_rate = 0.001

num_epochs = 100
batch_size = 128
learning_rate = 0.0001

In [8]:
%time model = MultiFullCAE(in_img_shape=(32,32), full_image_resize=(24,24)).cuda()

CPU times: user 1.15 s, sys: 316 ms, total: 1.46 s
Wall time: 1.46 s


In [9]:
# model.parameters

In [10]:
%%time
criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

CPU times: user 1.52 ms, sys: 723 µs, total: 2.24 ms
Wall time: 2.2 ms


In [11]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 32, 32)
    return x


In [12]:
%%time

transformation = monochrome_preprocess(32,32)
#train_loader, test_loader = get_loaders(batch_size, transformation, dataset=datasets.CocoDetection)
train_loader, test_loader = get_loaders(batch_size, transformation)

Files already downloaded and verified
CPU times: user 682 ms, sys: 205 ms, total: 887 ms
Wall time: 886 ms


In [13]:
%%time

for epoch in range(num_epochs):
    for i, (img, labels) in enumerate(train_loader):
        img = Variable(img).cuda()
        # ===================forward=====================
#         print("encoding batch of  images")
        output = model(img)
#         print("computing loss")
        loss = criterion(output, img)
        # ===================backward====================
#         print("Backward ")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        in_pic = to_img(img.cpu().data)
        save_image(pic, './fmrcae_results/in-64x64_32x32_3-5-7-11-out_image_{}.png'.format(epoch))
        save_image(in_pic, './fmrcae_results/in-64x64_32x32_3-5-7-11-in_image_{}.png'.format(epoch))
    if loss.data[0] < 0.15: #arbitrary number because I saw that it works well enough
        break

epoch [1/100], loss:0.3963


/home/leo/DeepLearning/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


FileNotFoundError: [Errno 2] No such file or directory: './fmrcae_results/in-64x64_32x32_3-5-7-11-out_image_0.png'

In [14]:
# torch.save(model, "fmrcae_in-64x64_32x32_3-5-7-11")

## Preliminary Results 

Experiments with the following configurations:


## Pytorch understanding tests